<pre>
Name: Ritika Sarkar
Reg. No.: 19BAI1031
</pre>

### SVM

In [1]:
# in google colab
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random
from google.colab.patches import cv2_imshow
import pandas as pd 
import os
from skimage.transform import resize
from skimage.io import imread

Kaggle API

In [2]:
mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

#### A. Linear SVM for [Social Network Ads dataset](https://www.kaggle.com/rakeshrau/social-network-ads)

##### Importing dataset

In [6]:
!kaggle datasets download rakeshrau/social-network-ads

  0% 0.00/3.27k [00:00<?, ?B/s]
100% 3.27k/3.27k [00:00<00:00, 5.82MB/s]


In [7]:
import zipfile as zp
zf = zp.ZipFile(r'social-network-ads.zip')
df1 = pd.read_csv(zf.open('Social_Network_Ads.csv'))

In [8]:
df1.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [9]:
from sklearn import preprocessing
# gender column label encoded to 0 1
le = preprocessing.LabelEncoder()
le.fit(df1.Gender)
df1['Gender'] = le.transform(df1.Gender)

In [10]:
df1.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,1,19,19000,0
1,15810944,1,35,20000,0
2,15668575,0,26,43000,0
3,15603246,0,27,57000,0
4,15804002,1,19,76000,0


In [11]:
df1.shape

(400, 5)

In [12]:
df1.isnull().sum()

User ID            0
Gender             0
Age                0
EstimatedSalary    0
Purchased          0
dtype: int64

In [13]:
# splitting to x and y
X = df1.drop(['Purchased'],axis = 1)
y = df1['Purchased']

In [14]:
# Splitting to train  
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [ ]:
# #Feature Scaling
# from sklearn.preprocessing import StandardScaler

# cols = X_train.columns
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

# X_train = pd.DataFrame(X_train,columns=cols)
# X_test = pd.DataFrame(X_test,columns=cols)

In [15]:
X_train.head()

,User ID,Gender,Age,EstimatedSalary
250,15810075,0,44,39000
63,15583681,1,32,120000
312,15733964,0,38,50000
159,15598840,0,32,135000
283,15663249,0,52,21000


In [16]:
X_train.dtypes

User ID            int64
Gender             int64
Age                int64
EstimatedSalary    int64
dtype: object

In [17]:
np.array([X_train])[0][1]

array([15583681,        1,       32,   120000])

In [18]:
np.array(y_train)

array([0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [31]:
# linear svm
svm1 = cv2.ml.SVM_create()
svm1.setType(cv2.ml.SVM_C_SVC) #n-class classification
svm1.setKernel(cv2.ml.SVM_LINEAR) # linearly sep data 
svm1.setTermCriteria((cv2.TERM_CRITERIA_MAX_ITER,500, 1e-6)) #termination criteria 500 iterations
svm1.train(np.asarray(X_train, np.float32), cv2.ml.ROW_SAMPLE, np.array(y_train))
# error solved using :-
# https://answers.opencv.org/question/203571/svm-train-error-215assertion-failed-problem/
# https://answers.opencv.org/question/137035/opencv-3-and-svm-on-cpp/ 

True

In [32]:
# PREDICT
response = svm1.predict(np.asarray(X_test, np.float32))[1]

In [33]:
sv = svm1.getUncompressedSupportVectors() # the support vectors
print(sv)

[[1.5810075e+07 0.0000000e+00 4.4000000e+01 3.9000000e+04]
 [1.5807525e+07 0.0000000e+00 5.9000000e+01 2.9000000e+04]
 [1.5815236e+07 0.0000000e+00 4.5000000e+01 1.3100000e+05]
 [1.5809823e+07 1.0000000e+00 2.6000000e+01 1.5000000e+04]
 [1.5813113e+07 1.0000000e+00 4.0000000e+01 1.0700000e+05]
 [1.5592570e+07 1.0000000e+00 4.7000000e+01 2.3000000e+04]
 [1.5665416e+07 0.0000000e+00 3.9000000e+01 7.1000000e+04]
 [1.5603942e+07 0.0000000e+00 5.1000000e+01 1.3400000e+05]
 [1.5571059e+07 0.0000000e+00 3.3000000e+01 4.1000000e+04]
 [1.5575694e+07 1.0000000e+00 3.5000000e+01 7.3000000e+04]
 [1.5575247e+07 0.0000000e+00 4.8000000e+01 1.3100000e+05]
 [1.5574372e+07 0.0000000e+00 5.8000000e+01 4.7000000e+04]
 [1.5573452e+07 0.0000000e+00 2.1000000e+01 1.6000000e+04]
 [1.5814816e+07 1.0000000e+00 3.1000000e+01 6.6000000e+04]
 [1.5611430e+07 0.0000000e+00 6.0000000e+01 4.6000000e+04]
 [1.5794661e+07 0.0000000e+00 2.6000000e+01 1.1800000e+05]]


In [34]:
# performance metrics
from sklearn.metrics import accuracy_score
print("Accuracy score", accuracy_score(y_test, [x[0] for x in response]))

Accuracy score 0.61


In [37]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix\n", confusion_matrix(y_test, [x[0] for x in response]))

Confusion matrix
 [[41 27]
 [12 20]]


In [46]:
from sklearn.metrics import classification_report
print(classification_report(y_test,[x[0] for x in response]))

              precision    recall  f1-score   support

           0       0.77      0.60      0.68        68
           1       0.43      0.62      0.51        32

    accuracy                           0.61       100
   macro avg       0.60      0.61      0.59       100
weighted avg       0.66      0.61      0.62       100



##### Observation
1. The low accuracy score on the test set, i.e. 61% shows that feature engineering and scaling is required. Also, presence of more data would be beneficial to the problem.
2. The effect of increasing or decreasing the number of iterations is seen as follows: any iteration value lesser or greater than 500 gives an accuracy value lower than 61%, which is quite interesting as it shows that 500 iterations is optimal for this dataset.
3. Also, we observe the support for class 1 is nearly half that of class 0. We find lower value of precision which means the classifier raises false positives more often, however the higher value of recall shows that most of class 1 samples are correctly predicted. We could try balancing the classes, and observe the results. 

References
1. https://docs.opencv.org/4.x/d1/d73/tutorial_introduction_to_svm.html